### Generative Adversarial Network 
### ************************************
#### 

#### GAN consist of two models that compete with each other to analyze, capture and copy the variations within a dataset
### Generator:
#### The generator in GAN learns to create fake data by incorporating feedback form the discriminator
### Discriminator:
#### The Discriminator in GAN is a classifier that identifies real data from the fake data created by the Generator


#### Mathematical Formula for working on GANs can be represented as:
#### $$V(D,G)=E_{x-Pdata(x)}[logD(x)]+E_{z-p(z)}[log(1-D(G(z))]$$
#### Where:
#### $G$=Generator
#### $D$=Discriminator
#### $Pdata(x)$=distribution of real data
#### $p(z)$=distribution of generator
#### $x$=sample from Pdata(x)
#### $z$=sample from P(z)
#### $D(x)$=Discriminator network
#### $G(z)$=Generator network

#### Steps for trainning GAN
#### -->Define the problem
#### --> Choose the architecture of GAN
#### --> Train Discriminator on real data
#### --> Generate fake data for Generator
#### --> Train Discriminator on fake data
#### --> Train Generator with the output of Discriminator

#### Simple GAN to generate points that follow normal distribution

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

# Define the generator network
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=100, activation='relu'))
    model.add(layers.Dense(2, activation='tanh'))  # Output is 2D for 2D points
    return model

# Define the discriminator network
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=2, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Build and compile the GAN
generator = build_generator()
z = tf.keras.Input(shape=(100,))
generated_image = generator(z)
discriminator.trainable = False
validity = discriminator(generated_image)
gan = tf.keras.Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Generate random data to train the GAN
def generate_real_data(n_samples):
    x = np.random.normal(0, 1, (n_samples, 2))
    y = np.ones((n_samples, 1))
    return x, y

def generate_fake_data(generator, n_samples):
    z = np.random.normal(0, 1, (n_samples, 100))
    x = generator.predict(z)
    y = np.zeros((n_samples, 1))
    return x, y

# Training parameters
batch_size = 64
epochs = 5000
sample_interval = 100

# Training loop
for epoch in range(epochs):
    real_data, real_labels = generate_real_data(batch_size)
    fake_data, fake_labels = generate_fake_data(generator, batch_size)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    valid_labels = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print proIn gress
    if epoch % sample_interval == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

        # Generate and save a sample of generated data
        if epoch % (sample_interval * 10) == 0:
            samples = generator.predict(np.random.normal(0, 1, (16, 100)))
            plt.scatter(samples[:, 0], samples[:, 1])
            plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
            plt.close()


2/2 [==============================] - 0s 3ms/step
Epoch 0, D Loss: 0.7003263235092163, G Loss: 0.6706808805465698
2/2 [==============================] - 0s 2ms/step
Epoch 100, D Loss: 0.7272081971168518, G Loss: 0.5128829479217529
2/2 [==============================] - 0s 3ms/step
Epoch 200, D Loss: 0.6883345544338226, G Loss: 0.6584524512290955
2/2 [==============================] - 0s 3ms/step
Epoch 300, D Loss: 0.7562277019023895, G Loss: 0.5873635411262512
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step
Epoch 400, D Loss: 0.7721287310123444, G Loss: 0.6651382446289062
2/2 [==============================] - 0s 2ms/step
Epoch 500, D Loss: 0.6306620836257935, G Loss: 0.7062184810638428
2/2 [==============================] - 0s 2ms/step
Epoch 600, D Loss: 0.5784453749656677, G Loss: 0.9293820858001709
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step
Epoch 700, D Loss: 0.535876139998436, G Loss: 1.0878322124481201
2/2 [==============================] - 0s 2ms/step
Epoch 800, D Loss: 0.6126810312271118, G Loss: 0.8168252110481262
2/2 [==============================] - 0s 996us/step
Epoch 900, D Loss: 0.6562667489051819, G Loss: 0.7772024869918823
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 3ms/step
Epoch 1000, D Loss: 0.6568450331687927, G Loss: 0.7283099889755249
2/2 [==============================] - 0s 996us/step
Epoch 1100, D Loss: 0.6329813599586487, G Loss: 0.7211847901344299
2/2 [==============================] - 0s 2ms/step
Epoch 1200, D Loss: 0.6387046575546265, G Loss: 0.708577573299408
2/2 [==============================] - 0s 997us/step


2/2 [==============================] - 0s 2ms/step
Epoch 1300, D Loss: 0.6547556519508362, G Loss: 0.7041370272636414
2/2 [==============================] - 0s 2ms/step
Epoch 1400, D Loss: 0.6404719948768616, G Loss: 0.7434285283088684
2/2 [==============================] - 0s 2ms/step
Epoch 1500, D Loss: 0.6406080722808838, G Loss: 0.7520751953125
2/2 [==============================] - 0s 997us/step


...

2/2 [==============================] - 0s 2ms/step
Epoch 3500, D Loss: 0.6090000867843628, G Loss: 0.8133241534233093
2/2 [==============================] - 0s 1ms/step
Epoch 3600, D Loss: 0.5836817026138306, G Loss: 0.878441333770752
2/2 [==============================] - 0s 2ms/step
Epoch 3700, D Loss: 0.5944784879684448, G Loss: 0.8511578440666199
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 998us/step
Epoch 3800, D Loss: 0.5909694135189056, G Loss: 0.839164137840271
2/2 [==============================] - 0s 2ms/step
Epoch 3900, D Loss: 0.6209841966629028, G Loss: 0.8134660720825195
2/2 [==============================] - 0s 2ms/step
Epoch 4000, D Loss: 0.5742009580135345, G Loss: 0.8563463687896729
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step
Epoch 4100, D Loss: 0.6192694902420044, G Loss: 0.8691353797912598
2/2 [==============================] - 0s 2ms/step
Epoch 4200, D Loss: 0.6048245131969452, G Loss: 0.865429699420929
2/2 [==============================] - 0s 2ms/step
Epoch 4300, D Loss: 0.6044120192527771, G Loss: 0.8703936338424683
2/2 [==============================] - 0s 2ms/step
Epoch 4400, D Loss: 0.5743887424468994, G Loss: 0.8311932682991028
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step
Epoch 4500, D Loss: 0.6181225478649139, G Loss: 0.8696799278259277
2/2 [==============================] - 0s 2ms/step
Epoch 4600, D Loss: 0.6643857657909393, G Loss: 0.8635481595993042
2/2 [==============================] - 0s 2ms/step
Epoch 4700, D Loss: 0.6047608554363251, G Loss: 0.8201195001602173
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step
Epoch 4800, D Loss: 0.6124545633792877, G Loss: 0.8379536867141724
2/2 [==============================] - 0s 2ms/step
Epoch 4900, D Loss: 0.6346192955970764, G Loss: 0.8548703193664551
2/2 [==============================] - 0s 1ms/step


#### Simple GAN to use generate digit objects

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

# Load and preprocess the MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 127.5 - 1.0  # Normalize to the range [-1, 1]
x_train = np.expand_dims(x_train, axis=-1)

# Define the generator network
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=100))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Define the discriminator network
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

# Build and compile the GAN
generator = build_generator()
z = tf.keras.Input(shape=(100,))
generated_image = generator(z)
discriminator.trainable = False
validity = discriminator(generated_image)
gan = tf.keras.Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Training parameters
batch_size = 64
epochs = 20000
sample_interval = 1000

# Training loop
for epoch in range(epochs):
    # Train the discriminator
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_images = x_train[idx]
    labels_real = np.ones((batch_size, 1))
    labels_fake = np.zeros((batch_size, 1))

    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_images, labels_real)
    d_loss_fake = discriminator.train_on_batch(fake_images, labels_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    labels_gan = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, labels_gan)

    # Print progress
    if epoch % sample_interval == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

        # Generate and save a sample of generated images
        if epoch % (sample_interval * 10) == 0:
            samples = generator.predict(np.random.normal(0, 1, (16, 100)))
            samples = 0.5 * samples + 0.5  # Rescale to [0, 1]
            fig, axs = plt.subplots(4, 4)
            count = 0
            for i in range(4):
                for j in range(4):
                    axs[i, j].imshow(samples[count, :, :, 0], cmap='gray')
                    axs[i, j].axis('off')
                    count += 1
            plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
            plt.close()

2/2 [==============================] - 0s 2ms/step
Epoch 0, D Loss: 0.664280354976654, G Loss: 0.6821396350860596
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step


...

2/2 [==============================] - 0s 4ms/step


2/2 [==============================] - 0s 3ms/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 3ms/step
